# 04_5 Linear Regression with Polynomial Features

Due to NDA agreements no data can be displayed.

Data Preparation, Data Cleaning, and Preparation for Modelling is the same for all algorithms. To directly go to modelling click [here](#modelling)

---

## Data preparation

### Import libraries and read data

In [ ]:
import pandas as pd 
import numpy as np

from sklearn.metrics import mean_squared_error 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.append("..")
import mlflow
from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

In [ ]:
# read data
df = pd.read_csv('../data/Featureselection03.csv')
df.head()

### Create data frame with important features

So that everyone is on track with the feature selection, we created another csv file to rate the importance and only use important features for training our models and further analysis.

Only important features are used to train the model. In this case we use 17 features beside the target.

In [ ]:
# read list with feature importance
data_log = pd.read_csv('../data/Capstone_features_Features.csv')
data_log.head()

In [ ]:
# create list of important features (feature importance < 3)
list_imp_feat = list(data_log[data_log['ModelImportance'] < 3]['VarName'])
len(list_imp_feat)

In [ ]:
df_model = df[list_imp_feat].copy()

In [ ]:
df_model.info()

### Fill and drop NaN

Values for V.SLPOG.act.PRC and ME.SFCI.act.gPkWh contain missing values. The EDA showed that these are mainly caused during harbour times when the main engine was not running. Therefore it makes sense to fill the missing values with 0.

In [ ]:
df_model['V.SLPOG.act.PRC'].fillna(0,inplace=True)
df_model['ME.SFCI.act.gPkWh'].fillna(0,inplace=True)

The remaining rows with missing values are dropped.

In [ ]:
df_model.dropna(inplace=True)

During further analysis one sample with an ME.FMS.act.tPh of 8 was defined as an outlier.

In [ ]:
df_model = df_model[df_model['ME.FMS.act.tPh'] < 8]
df_model.shape

In [ ]:
df_model.info()

### Check correlations

In [ ]:
plt.figure(figsize = (30,28))
sns.heatmap(df_model.corr(), annot = True, cmap = 'RdYlGn')

V.SOG.act.kn is still highly correlated with the target, but this feature is necessary to keep.

### Define target

For this project the focus is on optimising the fuel consumption. Therefore the supply mass rate is used as target. Target values greater 8 t/h are defined as outlier.

In [ ]:
X = df_model.drop(['ME.FMS.act.tPh'], axis = 1) #, 'Power_EM_predict', 'ME.SFCI.act.gPkWh'
y = df_model['ME.FMS.act.tPh']

### Train Test Split

Due to the high amount of data, a split into 10% test data and 90% train data is chosen. The random state is set to 42 to have comparable results for diffent models. To account for the imbalance in the distribution of passage types the stratify parameter is used for this feature. This results in approximately the same percentage of the different passage types in each subset.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = X['passage_type'], test_size = 0.2, random_state = 42)

### Create dummy values for passage type

Object types need to be transformed to dummy values. For this model this concerns the passage types.

In [ ]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

In [ ]:
X_train.info()

### Set MLFlow connection

MLFlow is used to track and compare different models and model settings.

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run(run_name='LinReg_with_PolyFeatures_no_EM_SFCI_>13.5kn') # CHANGE!
run = mlflow.active_run()

---

## Modelling

For the modeling a SGD Linear Regression was used in this notebook. Polynomial features of the second poylonmial were included as well as all polynomial features. This was done to be able to model higher polynomials and to include conections between the features.

For all models in this project a MinMaxScaler is applied. For this model a random forrest is used. The hyperparameter are selected based on grid search and offer a reasonable balance between optimal results and overfitting. These settings are used in a pipeline.

### Pipeline

In [ ]:
reg = make_pipeline(PolynomialFeatures(degree=2), MinMaxScaler() , LinearRegression()) # CHANGE!

### Fit and predict

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
y_pred = reg.predict(X_test)
y_pred_train = reg.predict(X_train)

With this model also negative values for the target are being predicted. This does not make sense. Therefore these values were set to 0.

In [ ]:
y_pred[y_pred < 0] = 0
y_pred_train[y_pred_train < 0] = 0

---

## Analysis

### Errors and residuals

The root mean squared error (RMSE) is used to evaluate the model. 

In [ ]:
print('RMSE train: ', mean_squared_error(y_train, y_pred_train, squared= False))
rmse_train = mean_squared_error(y_train, y_pred_train, squared= False)
print('RMSE test: ', mean_squared_error(y_test, y_pred, squared= False))
rmse_test = mean_squared_error(y_test, y_pred, squared= False)

Plotting actual values against predicted shows that the points are close to the optimal diagonale. However, this plot and the yellowbrick residual plot show some dificulties the model has when predicting low target values.

In [ ]:
fig=plt.figure(figsize=(6, 6))
plt.axline([1, 1], [2, 2],color='lightgrey')
plt.scatter(y_train, y_pred_train, color ='#33424F')
plt.scatter(y_test, y_pred, color = '#FF6600')
#plt.xticks(np.arange(0,501,100));
#plt.yticks(np.arange(0,501,100));
plt.xlabel("ME.FMS.act.tPh actual");
plt.ylabel("ME.FMS.act.tPh predicted");
plt.xlim(-1, 6);
plt.ylim(-1, 6);

Here it can be seen, that especially while trying to predict lower values for the target the model makes some errors in both directions. For higher values the model performs well.

---

### Write to MLFlow

In [ ]:
#seting parameters that should be logged on MLFlow
#these parameters were used in feature engineering (inputing missing values)
#or parameters of the model (fit_intercept for Linear Regression model)
params = {
      "features drop": 'according to Capstone_features_Features.csv + t/h kleiner 8 + EM and SCFI drop + Speed > 13.5 kn',
      "explanation": 'Linear Regression with Polynomial features of degree 2 and all negative Prediction set to 0',
      "csv used": 'Featureselection03.csv',
      "NaN handling": 'V.SLPOG.act.PRC and ME.SFCI.act.gPkWh filled with 0, rest dropped by row',
      'Shape' : df.shape,
      'Scaler' : 'MinMaxScaler',
      'Polynomialdegree' : 2,
      'size_test' : 0.2
  }

In [ ]:
#logging params to mlflow
mlflow.log_params(params)
#setting tags
mlflow.set_tag("running_from_jupyter", "True")
#logging metrics
mlflow.log_metric("train-" + "RMSE", rmse_train)
mlflow.log_metric("test-" + "RMSE", rmse_test)
# logging the model to mlflow will not work without a AWS Connection setup.. too complex for now
# but possible if running mlflow locally
# mlflow.log_artifact("../models")
# mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

## Further Analysis

### What are the most important features?

In [ ]:
X_train.columns

In [ ]:
print('Nr. of input features: ', reg.steps[0][1].n_features_in_)
print('Nr. of output features: ', reg.steps[0][1].n_output_features_)

16 features were input and due to the inclusion of polynomial features 153 were used for prediction. What are these features?

In [ ]:
reg.steps[0][1].get_feature_names_out()

The feature '1' at the beginning is the constant that explains the y axis section.

In [ ]:
reg.steps[2][1].coef_

What are the most important features for the model?

In [ ]:
df_coef = pd.DataFrame({'features' : reg.steps[0][1].get_feature_names_out(), 'coef' : reg.steps[2][1].coef_})
df_coef.sort_values('coef')

## Error Analysis

To get a better idea of the errors and understand the model better to be able to improve it further, an error analysis was performed.

In [ ]:
from yellowbrick.regressor import ResidualsPlot

The Residuals Plot from the yellowbrick package was used.

In the Residual plot the predicted values are ploted against the Residuals (Errors) to identify patterns were the models makes more or larger errors compared to other areas.

In [ ]:
visualizer = ResidualsPlot(reg)

visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show() 

Again it is observable that especially for smaller values larger errors are observable. The result where negative values were set to 0 can be seen below.

In [ ]:
residuals_test = y_pred - y_test
residuals_train = y_pred_train - y_train

In [ ]:
sns.scatterplot(x = y_pred_train, y = residuals_train)
sns.scatterplot(x = y_pred, y = residuals_test)
plt.axhline(y = 0, color = 'black')
plt.xlabel("ME.FMS.act.tPh predicted");
plt.ylabel("Residuals");
plt.legend(labels=['', 'train', 'test'])

Compared with the other plots it is observable that the model makes the most errors when predicting values that are actual 0 but the model predicts a higher value.

## Understanding what happens when the t/h is 0 but values are getting predicted

To analyse why the model has trouble with these values, these values were analysed further.

In [ ]:
X_train_actzeros = X_train.copy()
X_train_actzeros['actual'] = y_train
X_train_actzeros['predict'] = y_pred_train

Data points that are actual 0 values but the model predicts with more than 0.1 (little below the RMSE) are problematic.

In [ ]:
X_train_actzeros = X_train_actzeros[X_train_actzeros['actual'] == 0]
X_train_actzeros = X_train_actzeros[X_train_actzeros['predict'] > 0.1]
X_train_actzeros

In [ ]:
X_train_actzeros.columns

In [ ]:
sns.pairplot(data=X_train_actzeros,
                  x_vars=['predict'],
                  y_vars=['ME.FTS.act.dgC', 'HFO.GME.act.nodim', 'V.COG.act.deg', 'V.SOG.act.kn',
       'V.RUA.act.deg','WEA.WDT.act.deg', 'WEA.WST.act.mPs', 'V.SLPOG.act.PRC',
        'DDM.TRIM.act.m', 'DDM.DRAFT.act.m',
       'Wave_Height_m_daily', 'True_Wave_Dir_deg_daily'])

The most promising was the V.SOG.act.kn. Therefore it was analysed more closely.

In [ ]:
import plotly.express as px

In [ ]:
px.scatter(X_train_actzeros, x='V.SOG.act.kn',
            y='predict',
            color='predict',
            hover_data=['V.RUA.act.deg'],
            labels={'x':'V.SOG.act.kn','y':'predict','color':'V.RUA.act.deg'})

Most of these datapoints are at a speed between 0 and 10 kn. Some of them could be explained by being towed by a tugboat. No data was given wether a tugboat was used. Therefore it is difficult to analyse further.

## Analysing points with high residuals

In [ ]:
X_train_res = X_train.copy()
X_train_res['actual'] = y_train
X_train_res['predict'] = y_pred_train
X_train_res['residuals'] = residuals_train
X_train_res.head()

Datapoints with an error of +1 or -1 were analysed.

In [ ]:
X_train_res = X_train_res[(X_train_res['residuals'] > 1) | (X_train_res['residuals'] <  -1)]

In [ ]:
sns.pairplot(data=X_train_res,
                  x_vars=['predict'],
                  y_vars=['ME.FTS.act.dgC', 'HFO.GME.act.nodim', 'V.COG.act.deg', 'V.SOG.act.kn',
       'V.RUA.act.deg','WEA.WDT.act.deg', 'WEA.WST.act.mPs', 'V.SLPOG.act.PRC',
        'DDM.TRIM.act.m', 'DDM.DRAFT.act.m',
       'Wave_Height_m_daily', 'True_Wave_Dir_deg_daily'])

The most promising feature was again the V.SOG.act.kn

In [ ]:
px.scatter(X_train_res, x='actual',
            y='predict',
            color='V.SOG.act.kn',
            hover_data=['V.RUA.act.deg'],
            labels={'x':'V.SOG.act.kn','y':'predict','color':'V.RUA.act.deg'})

No possible pattern could be seen. The datapoints are distributed without a recognizable pattern in the color.